In [1]:
words = open('names.txt','r').read().splitlines()

In [2]:
import torch
import torch.nn.functional as F
torch.set_default_device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

In [4]:
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        # print(ch1, ch2)
        xs.append([ix1, ix2])
        ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.size(0)
print('number of examples: ',num)

number of examples:  196113


In [5]:
xs

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        ...,
        [26, 25],
        [25, 26],
        [26, 24]], device='cuda:0')

In [6]:
ys

tensor([13, 13,  1,  ..., 26, 24,  0], device='cuda:0')

In [20]:
g = torch.Generator().manual_seed(2147483647)
xenc = F.one_hot(xs, num_classes=27).float()
xenc = xenc.view(-1,2*27)
xenc.shape

torch.Size([196113, 54])

In [21]:
W = torch.randn((2*27, 27), requires_grad=True)

In [24]:
for k in range(500):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    
    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

2.2525792121887207
2.2525784969329834
2.252577543258667
2.2525768280029297
2.2525758743286133
2.252574920654297
2.2525742053985596
2.2525734901428223
2.252572536468506
2.2525718212127686
2.252570867538452
2.252570390701294
2.2525691986083984
2.2525687217712402
2.252568006515503
2.2525670528411865
2.252566337585449
2.252565383911133
2.2525649070739746
2.252563953399658
2.252563238143921
2.2525622844696045
2.2525618076324463
2.25256085395813
2.2525601387023926
2.2525594234466553
2.252558708190918
2.2525577545166016
2.2525572776794434
2.252556324005127
2.2525556087493896
2.2525546550750732
2.252553939819336
2.2525532245635986
2.2525522708892822
2.252551794052124
2.2525508403778076
2.2525501251220703
2.252549409866333
2.2525486946105957
2.2525479793548584
2.2525475025177
2.252546548843384
2.2525455951690674
2.25254487991333
2.2525441646575928
2.2525434494018555
2.252542734146118
2.25254225730896
2.2525413036346436
2.2525408267974854
2.252540111541748
2.2525393962860107
2.2525386810302734
2

In [25]:
import random

In [26]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix1 = 0
    ix2 = random.randint(1,26)
    out.append(itos[ix2])

    while True:
        xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
        logits = xenc.view(-1,2*27) @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        ix1 = ix2
        ix2 = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itos[ix2])
        if ix2 == 0:
            break
    print(''.join(out))

milekahe.
phrie.
uvi.
nadiyamagrax.
urenj.
